In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import glob

# Import Dataset

In [3]:
os.getcwd()

'C:\\Users\\ill will\\Desktop\\Code\\springboard\\Capstone_1\\notebooks'

In [4]:
import importlib.util
import sys

current_file_path = os.path.abspath(os.path.join("__file__" ,"../.."))
current_file_path

'C:\\Users\\ill will\\Desktop\\Code\\springboard\\Capstone_1'

In [5]:
import_data_path = os.path.join(current_file_path,'src','data')
sys.path.append(os.path.abspath(os.path.join(import_data_path)))
import prep_data_for_models as pdfm

import_eda_path = os.path.join(current_file_path,'src','tools')
sys.path.append(os.path.abspath(os.path.join(import_eda_path)))
import eda_tools as eda

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [6]:
data = pdfm.get_data()

In [7]:
movies = pdfm.get_movies(data)

In [8]:
data.keys()

dict_keys(['tmdb_cast_credit', 'tmdb_crew_credit', 'tmdb_movie_genres', 'tmdb_movie_keywords', 'tmdb_movie_main', 'tmdb_movie_production_companies', 'tmdb_movie_production_countries', 'tmdb_movie_spoken_languages'])

In [9]:
actors_df = pdfm.get_actor_credits(data, movies, False, False)

getting cast credits to date


HBox(children=(IntProgress(value=0, max=1811), HTML(value='')))


getting cast net to date


HBox(children=(IntProgress(value=0, max=1811), HTML(value='')))

In [10]:
actors_df.rename(columns={'gender': 'actor_gender','credits': 'actor_credits', 'net_to_date':'actor_net_to_date'}, inplace=True)

In [11]:
crew_dfs = pdfm.get_crew_credits(data, movies, False, False)

Producer


HBox(children=(IntProgress(value=0, max=1592), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1592), HTML(value='')))


Writer


HBox(children=(IntProgress(value=0, max=1092), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1092), HTML(value='')))


Director


HBox(children=(IntProgress(value=0, max=2018), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2018), HTML(value='')))

In [12]:
for k, df in crew_dfs.items():
    crew_dfs[k].rename(columns={'gender':k.lower() + '_gender', 'credits':k.lower() + '_credits', 'net_to_date': k.lower() + '_net_to_date'}, inplace=True)

In [13]:
mega_df = actors_df.drop('name', axis=1)
for k,df in crew_dfs.items():
    mega_df = mega_df.merge(df.drop('name',axis=1), on=['movie_id', 'title'], how='inner')

In [14]:
genres_df = pdfm.get_genre_data(data, movies, False, False).reset_index()

In [15]:
mega_df = mega_df.merge(genres_df, on=['movie_id', 'title'])

In [16]:
lang_df = pdfm.get_language_data(data, movies, False, False).reset_index()

In [17]:
mega_df = mega_df.merge(lang_df, on=['movie_id', 'title'])

In [18]:
mega_df.head()

movie_id               title  actor_gender  actor_credits  \
0         5          Four Rooms             2              0   
1        11           Star Wars             2              0   
2        16  Dancer in the Dark             1              0   
3        24   Kill Bill: Vol. 1             1              0   
4        28      Apocalypse Now             2              0   

   actor_net_to_date  producer_gender  producer_credits  producer_net_to_date  \
0                  0                2                 2             219389769   
1                  0                2                 1             139223000   
2                  0                0                 0                     0   
3                  0                2                 7             518392527   
4                  0                2                 2              37362841   

   writer_gender  writer_credits  writer_net_to_date  director_gender  \
0              1               0                   0                1   
1              2               0                   0                2   
2              2               0                   0                2   
3              2               1            13461007                2   
4              2               0                   0                2   

   director_credits  director_net_to_date  genre_count  language_count  
0                 0                     0            2               1  
1                 1             139223000            3               1  
2                 0                     0            3               1  
3                 3             247062931            2               3  
4                 3             276429252            2               3

In [19]:
movies = movies[['budget','revenue','release_date','runtime']].reset_index()

In [20]:
movies.head()

movie_id                                     title     budget     revenue  \
0     19995                                    Avatar  237000000  2787965087   
1       285  Pirates of the Caribbean: At World's End  300000000   961000000   
2    206647                                   Spectre  245000000   880674609   
3     49026                     The Dark Knight Rises  250000000  1084939099   
4     49529                               John Carter  260000000   284139100   

  release_date  runtime  
0   2009-12-10      162  
1   2007-05-19      169  
2   2015-10-26      148  
3   2012-07-16      165  
4   2012-03-07      132

In [21]:
movies = eda.split_release_date(movies)

In [22]:
mega_df = pd.merge(movies.drop('release_date', axis=1),mega_df, on=['movie_id','title'])

In [23]:
mega_df.set_index(['movie_id', 'title'], inplace=True)

In [24]:
mega_df['net'] = mega_df['revenue'] - mega_df['budget']

In [25]:
mega_df['target'] = mega_df['net'].apply(lambda x: 1 if x > 0 else 0)

In [26]:
mega_df.head()

budget     revenue  \
movie_id title                                                              
19995    Avatar                                     237000000  2787965087   
99861    Avengers: Age of Ultron                    280000000  1405403694   
41154    Men in Black 3                             225000000   624026776   
122917   The Hobbit: The Battle of the Five Armies  250000000   956019788   
37724    Skyfall                                    200000000  1108561013   

                                                    runtime  release_month  \
movie_id title                                                               
19995    Avatar                                         162             12   
99861    Avengers: Age of Ultron                        141              4   
41154    Men in Black 3                                 106              5   
122917   The Hobbit: The Battle of the Five Armies      144             12   
37724    Skyfall                                        143             10   

                                                    release_year  release_dow  \
movie_id title                                                                  
19995    Avatar                                             2009            3   
99861    Avengers: Age of Ultron                            2015            2   
41154    Men in Black 3                                     2012            2   
122917   The Hobbit: The Battle of the Five Armies          2014            2   
37724    Skyfall                                            2012            3   

                                                    actor_gender  \
movie_id title                                                     
19995    Avatar                                                2   
99861    Avengers: Age of Ultron                               2   
41154    Men in Black 3                                        2   
122917   The Hobbit: The Battle of the Five Armies             2   
37724    Skyfall                                               2   

                                                    actor_credits  \
movie_id title                                                      
19995    Avatar                                                 0   
99861    Avengers: Age of Ultron                               11   
41154    Men in Black 3                                        12   
122917   The Hobbit: The Battle of the Five Armies              2   
37724    Skyfall                                                6   

                                                    actor_net_to_date  \
movie_id title                                                          
19995    Avatar                                                     0   
99861    Avengers: Age of Ultron                           3940617206   
41154    Men in Black 3                                    3150078356   
122917   The Hobbit: The Battle of the Five Armies          762878416   
37724    Skyfall                                            955438019   

                                                    producer_gender   ...    \
movie_id title                                                        ...     
19995    Avatar                                                   2   ...     
99861    Avengers: Age of Ultron                                  2   ...     
41154    Men in Black 3                                           2   ...     
122917   The Hobbit: The Battle of the Five Armies                2   ...     
37724    Skyfall                                                  1   ...     

                                                    writer_gender  \
movie_id title                                                      
19995    Avatar                                                 2   
99861    Avengers: Age of Ultron                                2   
41154    Men in Black 3                                         0   
122917   The Hobbit: The Bat

In [27]:
mega_df.to_csv('mega_features.csv')